In [1]:
from IPython.display import display, Image, clear_output, HTML
import time
import random 
random.seed(1)
import ipywidgets as widgets
from jupyter_ui_poll import ui_events
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import json
import re

In [2]:
results_dict = {
    'filename': [],
    'nL': [],
    'nR': [],
    'correct': [],
    'display_duration': []
}

In [3]:
def register_btn_event(btn):
    event_info['type'] = "button click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return

In [4]:
def display_test_img(img_file):
    style_str = f'width: 600px;'
    html_out = HTML(f"<img style='{style_str}' src={img_file}></img>")
    display(html_out)
    
display_test_img("myfigure.png") 

In [5]:
def send_to_google_form(data_dict, form_url):
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

In [6]:
event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

# this function lets buttons 
# register events when clicked
def register_btn_event(btn):
    global start_clicked
    if btn.description == "Start":
        start_clicked = True
    else:
        event_info['type'] = "button click"
        event_info['description'] = btn.description
        event_info['time'] = time.time()
    return  

In [7]:
def run_ans_test(img_file, right_answer, display_duration):
    blank_pic = Image("blank.png", width=600)

    btn1 = widgets.Button(description="blue")
    btn2 = widgets.Button(description="yellow")

    btn1.on_click(register_btn_event)
    btn2.on_click(register_btn_event)

    display_test_img(img_file)

    time.sleep(display_duration)
    clear_output(wait=True)
    display(blank_pic)
    print("Input your answer: blue or yellow")
    display(btn1)
    display(btn2)

    result = wait_for_event(timeout=3)

    if result['description'] == right_answer:
        score = 1
    else:
        score = 0

    results_dict['filename'].append(img_file)

    match = re.match(r'(\d+)_vs_(\d+)', img_file)
    if match:
        nL, nR = match.groups()
        results_dict['nL'].append(int(nL))
        results_dict['nR'].append(int(nR))
    else:
        results_dict['nL'].append(None)
        results_dict['nR'].append(None)

    results_dict['display_duration'].append(display_duration)
    
    if result['description'] == right_answer:
        results_dict['correct'].append('yes')
    else:
        results_dict['correct'].append('no')

    clear_output(wait=True)
    return score


In [8]:
def run_ans_full(num_repeats):
    files = ["12_vs_14.png", "12_vs_16.png", "15_vs_20.png", "16_vs_18.png", "18_vs_20.png", "18_vs_21.png", "9_vs_10.png", "9_vs_12.png", "10_vs_9_b.png", "12_vs_9_b.png", "14_vs_12_b.png", "16_vs_12_b.png", "18_vs_16_b.png", "20_vs_15_b.png", "20_vs_18_b.png", "21_vs_18_b.png"]
    answers = ["yellow", "yellow", "yellow", "yellow", "yellow", "yellow", "yellow", "yellow", "blue", "blue", "blue", "blue", "blue", "blue", "blue", "blue"]    
    
    total = 0
    score = []
    

    data_consent_info = """DATA CONSENT INFORMATION:

Please read:

we wish to record your response data

to an anonymised public data repository.

Your data will be used for educational teaching purposes

practising data analysis and visualisation.

Data collected will be completely anonymised and will include

gender identity and age.

Please type 'yes' in the box below if you consent to the upload."""

    print(data_consent_info)

    result = input("> ")

    if result.lower() == "yes":

        print("Thanks for your participation.")

        print("Please contact philip.lewis@ucl.ac.uk")

        print("If you have any questions or concerns")

        print("regarding the stored results.")
        
        time.sleep(3)
        clear_output(wait=True)
    else:

        raise(Exception("User did not consent to continue test."))

        
    print("""
Welcome to the ANS test!
For each trial, click the button which corresponds to the colour of the circles in the oval with the greatest number of circles, 
but be quick as you only have 3 seconds to answer!
You will not be told whether your answer is right or wrong.
The test will take a total of approximately 5 minutes.
""") 
    print("""
To generate your unique ID, please input the first two letters of the first name of a childhood best friend, 
and the initial of your favourite actor. For example, Amy and Tom Cruise would give the ID AMTC
""")
    ans1 = input(">> ")
    print("What is your age in years? please give a number:")
    ans2 = input(">> ")
    print("What gender do you identify as? please input f for female, m for male, nb for nonbinary, o for other, and pns for prefer not to say: ")
    ans3 = input(">> ")
    clear_output(wait=True)

    print("""
The test will begin shortly!
Please click on the corresponding button to the colour of the circles within the oval containing
the highest number of circles. 
You only have 3 seconds to input your answer, and it will automatically continue onto the next
trial after 3 seconds.
""")
    input("Press Enter to continue...")
    
    clear_output(wait=True)

    print("Get ready to start the test!")

    display_durations = [0.5] * (num_repeats // 2) + [0.75] * (num_repeats // 2)
    random.shuffle(display_durations)

    for countdown in range(3, 0, -1):
        print(countdown)
        time.sleep(1)

    start_button = widgets.Button(description="Start")
    start_button.on_click(register_btn_event)

    display(start_button)

    start_clicked = False

    wait_for_event(timeout=3)

    time.sleep(2)

    clear_output(wait=True)

    for i in range(num_repeats):
        indices = list(range(len(files)))
        random.shuffle(indices)

        for j in indices:
            score = run_ans_test(files[j], answers[j], display_duration=display_durations[i])
            total += score
        
    df = pd.DataFrame(results_dict)

    data_dict = {
        'id': ans1,
        'age': ans2,
        'gender': ans3,
        'score': total,
        'results': df.to_json()
    }

    form_url = "https://docs.google.com/forms/d/e/1FAIpQLScVS__ITc_Ju4ASqFvF-i14lLPBEdFCqLk-vU4RqIkBrK7VRw/viewform?usp=sf_link"
    send_to_google_form(data_dict, form_url)
           
    print("You scored", total)
    print("""
Thank you for completing the ANS test. 
The ANS test is the Approximate Number System test. It is a cognitive 
test that assesses estimation of numerosities (i.e. a set of items) without
the use of language. 
It does not serve as a clinical cognitive assessment in any capacity. 
This research is being done purely to investigate individual differences in 
numerosity estimation acuity.
If you have any concerns or questions regarding the task you
have participated in today, please contact 
maia.armstrong.20@ucl.ac.uk

Please contact philip.lewis@ucl.ac.uk 
for any concerns regarding data protection.
""")
    

num_repeats = 8
data_dict = run_ans_full(num_repeats)


You scored 112

Thank you for completing the ANS test. 
The ANS test is the Approximate Number System test. It is a cognitive 
test that assesses estimation of numerosities (i.e. a set of items) without
the use of language. 
It does not serve as a clinical cognitive assessment in any capacity. 
This research is being done purely to investigate individual differences in 
numerosity estimation acuity.
If you have any concerns or questions regarding the task you
have participated in today, please contact 
maia.armstrong.20@ucl.ac.uk

Please contact philip.lewis@ucl.ac.uk 
for any concerns regarding data protection.

